In [11]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [12]:
# Data extraction statements
egfr_statement = r'(?i)\[OrderWise Additional Information\] eGFR: (?:> )?(\d*) ml'
preg_statement = r'(?i)\[Pregnant\] (.*)\n'
suspected_cancer_statement = r'(?i)\[Suspected Cancer Referral\] (.*)\n'
allergies_statement = r'(?i)\[(?:Patient )?Allergies\] (.*)'
primary_reason_statement = r'(?i)\[OrderWise Primary Reason\] (?:Custom \()?([a-zA-Z\s,\/-]+)(?:\()?'
clarifications_statement = r'(?i)\[OrderWise Clarifications\] (.*)\n'
requested_by_consultant_statement = r'(?i)\[Scan Requested by Consultant\?\] (.*)\n'

requesting_consultant_statement = r'(?i)(| Requesting Consultant: .*)'
telephone_number_statement = r'(?i)\[Bleep/Telephone Number\] (.*)'
mobile_number_statement = r'(?i)(07\d{9}|447\d{9})'
order_comms_statement = r'(?i)~\(Information via Order Comms\)'

# Data cleaning statements
rem_order_comms_statement = r'(?i)\n~[\(\[]Information via Order Comms[\)\]]'
rem_orderwise_statement = r'(?i)(\n)?\[OrderWise Additional Information\].*\d{4}-\d{2}-\d{2}( |)?'
rem_orderwise_statement_eGFR = r'(?i)(\n)?\[OrderWise Additional Information\].*m\*?2( |)?'
rem_follow_statement = r'(?i)F/U date.*::(\s+\d{1,2}(\/|.|-)\d{1,2}(\/|.|-)\d{2,4})?'
rem_special_statement = r'[!,*[\]()&%|~+-]'
rem_new_line_statement = r'\n'
rem_date_statement = r'\d{2}[/.]\d{2}[/.]\d{2}[/.]'

preparation_statement = r'(?i)RUH (.*)\n'

# Data analysis statements
count_results_statement = r'(?i)type: (\D*),'

In [13]:
def get_data(statement, x):
    result = re.findall(statement, x)
    result = ",".join(set(result))
    if result == '' or result is None:
        return 'None'
    else:
        return result


def clean_column(column, statement, replace=''):
    df[column] = df[column].str.replace(statement, replace, regex=True)


def get_full_results(results):
    recognizer_list = [x.recognizer_results for x in results]
    return recognizer_list


def count_results(results):
    recognizer_list = [x.recognizer_results for x in results]
    result = re.findall(count_results_statement, recognizer_list.__str__())
    return result


def count_remove_tag(tag_name):
    count_col = 0
    columns = ['Clinical history', 'Comment', 'Preparation',]
    for column in columns:
        count_col += df[column].str.count(tag_name).sum()
        clean_column(column, tag_name)
    return count_col


def encode_drop(dataframe):
    ohe_columns = ['eGFR', 'Referring Locat', 'Speciality', 'Exam', 'Sex', 'Pregnancy Status', 'Suspected Cancer', 'Allergies', 'Consultant Request']
    le_columns = ['Protocol']

    le = LabelEncoder()
    for col in le_columns:
        dataframe[col] = le.fit_transform(dataframe[col])

    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)
    one_hot_ft = ohe.fit_transform(dataframe[ohe_columns])
    one_hot_df = pd.DataFrame(one_hot_ft, columns=ohe.get_feature_names_out(), index=dataframe.index)
    dataframe = pd.concat([dataframe, one_hot_df], axis=1)

    dataframe = dataframe.drop(['eGFR', 'Referring Locat', 'Speciality', 'Exam', 'Sex', 'Pregnancy Status',
              'Suspected Cancer', 'Allergies', 'Consultant Request'], axis=1)

    return dataframe

In [14]:
'''
1. IMPORT EXCEL DATA

Please replace file_path with the path to the folder containing the dataset file.
set_sample = True if using sample dataset only
'''
# Import excel file
set_sample = True

file_path = '/content/drive/MyDrive/Colab Notebooks/Final Notebooks/Submission/'

if set_sample:
    file_name = 'Data/SampleRaw.xlsm'
else:
    file_name = 'Data/FullRawData.xlsm'

df = pd.read_excel(file_path + file_name)

# Set data augmentation boolean
aug_data = True

# Set split ratio (train %)
split_ratio = 0.7
file_label = str(int(split_ratio*100)) + '-' + str(round((1-split_ratio)*100))

# Remove anonymised tags
print(f"<REM> total: {count_remove_tag('<REM>')}")
print(f"<REQ> total: {count_remove_tag('<REQ>')}")
print(f"<MNAME> total: {count_remove_tag('<MNAME>')}")
print(f"<EMAIL> total: {count_remove_tag('<EMAIL>')}")
print(f"<NUM> total: {count_remove_tag('<NUM>')}")
print(f"<ADDRESS> total: {count_remove_tag('<ADDRESS>')}")
print(f"<REP> total: {count_remove_tag('<REP>')}")


<REM> total: 198
<REQ> total: 99
<MNAME> total: 16
<EMAIL> total: 0
<NUM> total: 37
<ADDRESS> total: 0
<REP> total: 39


In [15]:
'''
2. EXTRACT COMMENT DATA INTO NEW COLUMNS

COMMENT COLUMN
This column contains semi-structured text data, for the requesting clinician to provide additional clinical information.
Note not all these boxes are mandatory to fill depending on the patient, e.g. pregnancy.
Entries thought to be potentially relevant are extracted to a separate column, but not others e.g. [Scan Requested
by Consultant Y/N]
'''
# Extract eGFR
df['eGFR'] = df['Comment'].str.extract(egfr_statement).fillna('None')

# Extract Pregnancy status
df['Pregnant'] = df['Comment'].str.extract(preg_statement).fillna('None')

# Extract Suspected cancer referral status
df['Suspected Cancer'] = df['Comment'].str.extract(suspected_cancer_statement).fillna(0)

# Extract Allergy
df['Allergies'] = np.array(get_data(allergies_statement, str(x)) for x in df['Comment'])

# Extract Primary Reason, remove irrelevant markers
df['Primary Reason'] = np.array(get_data(primary_reason_statement, str(x)) for x in df['Comment'])
df['Primary Reason'] = df['Primary Reason'].str.replace(' (RUHB)', '')
df['Primary Reason'] = df['Primary Reason'].str.replace(' (iRefer)', '')
df['Primary Reason'] = df['Primary Reason'].str.replace('None', '')

# Extract Scan Requested by Consultant
df['Consultant Request'] = df['Comment'].str.extract(requested_by_consultant_statement)


In [16]:
'''
3. SECONDARY PII CHECK AND DELETION
Remove any structured entries that could contain PII
Note this is a secondary check, all data should be fully de-identified by this point.
'''

# Remove telephone numbers
clean_column('Clinical history', mobile_number_statement)
clean_column('Clinical history', telephone_number_statement)

# Remove requesting consultant statement
clean_column('Clinical history', requesting_consultant_statement)

In [17]:
'''
4. FORMAT COLUMNS
Remove extraneous information and put data into appropriate format
'''
yes_list = ['Y', 'YES']
no_list = ['N', 'NO']
unknown_list = ['unknown', 'don\'t know']

# Clinical history
clean_column('Clinical history', rem_order_comms_statement)
clean_column('Clinical history', rem_orderwise_statement)
clean_column('Clinical history', rem_orderwise_statement_eGFR)
clean_column('Clinical history', rem_follow_statement)
clean_column('Clinical history', rem_special_statement, ' ')
clean_column('Clinical history', rem_new_line_statement, ' ')

df['Clinical history'] = df['Primary Reason'] + ' ' + df['Clinical history']

# eGFR - convert to categorical values based on stage CKD
df['eGFR'] = df['eGFR'].replace('None', 0)
df['eGFR'] = df['eGFR'].astype(float)
df['eGFR']= pd.cut(df['eGFR'],[-1, 0, 14, 29, 44, 59, 89, 100], labels=['None', '5', '4', '3b', '3a', '2', '1'])

# Suspected Cancer
df['Suspected Cancer'] = df['Suspected Cancer'].replace(['(?i)' + x for x in yes_list], 1, regex=True)
df['Suspected Cancer'] = df['Suspected Cancer'].replace(['(?i)' + x for x in no_list], 0, regex=True)

# Allergies - 0:No, 1:Yes, 2:Unknown
df['Allergies'] = df['Allergies'].replace(['(?i)' + x for x in unknown_list], 2, regex=True)
df['Allergies'] = df['Allergies'].replace('(?i)yes', 1, regex=True)
df['Allergies'] = df['Allergies'].replace('(?i)no', 0, regex=True)

# Urgency - Non-urgent (1,2,3) and Urgent (4,5,6)
df['UrgBin'] = np.where(df['Urg'] > 3, 1, 0)

# Pregnancy
df['Pregnancy Status'] = df.apply(lambda x: 'N' if x['Sex'] == 'M' else x['Pregnant'], axis=1)

# Preparation: the protocol chosen by the vetting consultant radiologist.
df['Prep'] = np.array(get_data(preparation_statement, str(x)) for x in df['Preparation'])
df['Protocol'] = df['Prep'].astype(str)
df['Protocol'] = df['Protocol'].str.lower()
df.drop(df.loc[df['Protocol']=='none'].index, inplace=True)
df.drop(df.loc[df['Protocol']=='none,none'].index, inplace=True)

# Age - normalised
scaler = MinMaxScaler()
df['Age'] = scaler.fit_transform(df[['Age At Event']])
df.drop(['Age At Event'], axis=1, inplace=True)

df = df.drop(['Event Date', 'Comment', 'Primary Reason',
              'Preparation', 'Pregnant', 'Prep'], axis=1)

In [18]:
'''
5. ENCODE CATEGORICAL DATA AND CREATE LABEL SET
'''
df = encode_drop(df)
y = df['Protocol']

df.to_csv(file_path+'AllData'+file_label+'.csv', encoding='utf-8', index=False)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [19]:
'''
6. SPLIT TRAINING DATA FOR AUGMENTATION

Note augmentation on a separate notebook due to time and memory constraints.
For the smaller example dataset Sample.xlsm, split stratification has been removed (stratify=y)
'''
if set_sample:
    train_data, test_data, y_train, y_test = train_test_split(df, y, test_size=1-split_ratio, random_state=42)
else:
    train_data, test_data, y_train, y_test = train_test_split(df, y, test_size=1-split_ratio, random_state=42, stratify=y)

if aug_data:
    train_data.to_csv(file_path + "TrainData"+ file_label +".csv", encoding='utf-8', index=False)
    test_data.to_csv(file_path + "TestData"+ file_label +".csv", encoding='utf-8', index=False)



In [20]:
'''
7. CREATE NON-AUGMENTED DATASETS
'''
# Extract text and labels from datasets and split validation/test data
labels = ['Protocol', 'Urg', 'UrgBin']

y_train = train_data[labels]
y_test = test_data[labels]

train_data.to_csv(file_path + "Xtrain_"+ file_label +"_no-aug.csv", encoding='utf-8', index=False)
test_data.to_csv(file_path + "Xtest_"+ file_label +"_no-aug.csv", encoding='utf-8', index=False)
y_train.to_csv(file_path + "ytrain_"+ file_label +"_no-aug.csv", encoding='utf-8', index=False)
y_test.to_csv(file_path + "ytest_"+ file_label +"_no-aug.csv", encoding='utf-8', index=False)
